<a href="https://www.kaggle.com/code/klyushnik/insurance-cross-selling-full-project?scriptVersionId=188342073" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl
mpl.rcParams.update(mpl.rcParamsDefault)
import warnings
warnings.filterwarnings('ignore')
from IPython.display import Image
%matplotlib inline

# from ydata_profiling import ProfileReport

from sklearn.model_selection import (cross_val_score, train_test_split,
                                     StratifiedKFold, 
                                     KFold, StratifiedGroupKFold)#
from sklearn import metrics
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.preprocessing import (QuantileTransformer, StandardScaler, 
                                   PowerTransformer, MaxAbsScaler,
                                  MinMaxScaler, RobustScaler,LabelEncoder,OneHotEncoder,
                                   StandardScaler, label_binarize, PolynomialFeatures)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import VarianceThreshold

from imblearn.over_sampling import SMOTE,RandomOverSampler
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, initializers, regularizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Layer
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from tensorflow.keras.initializers import Constant
from tensorflow.keras.optimizers import Adam
#from tensorflow_addons.optimizers import AdamW
from catboost import CatBoostClassifier
import lightgbm as lgb

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e7/sample_submission.csv
/kaggle/input/playground-series-s4e7/train.csv
/kaggle/input/playground-series-s4e7/test.csv


2024-07-15 07:05:16.407453: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-15 07:05:16.407611: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-15 07:05:16.588518: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Insurance Cross Selling

* ![64e3afacc1f35831121d20e8_20230821T0632-856acb87-f370-4ac8-8eab-6b46bb970dac.jpeg](https://cdn.prod.website-files.com/5e6aa7798a5728055c457ebb/64e3afacc1f35831121d20e8_20230821T0632-856acb87-f370-4ac8-8eab-6b46bb970dac.jpeg)

**The objective of this competition is to predict which customers respond positively to an automobile insurance offer.**

**Submissions are evaluated using area under the ROC curve using the predicted probabilities and the ground truth targets.**
![ROC-curves-and-area-under-curve-AUC.png](https://www.researchgate.net/publication/351506473/figure/fig2/AS:1022506102169601@1620795860506/ROC-curves-and-area-under-curve-AUC.png)

In [2]:
train = pd.read_csv("/kaggle/input/playground-series-s4e7/train.csv")
test = pd.read_csv("/kaggle/input/playground-series-s4e7/test.csv")
print(f'Shape of our data:{train.shape, test.shape}')

Shape of our data:((11504798, 12), (7669866, 11))


In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11504798 entries, 0 to 11504797
Data columns (total 12 columns):
 #   Column                Dtype  
---  ------                -----  
 0   id                    int64  
 1   Gender                object 
 2   Age                   int64  
 3   Driving_License       int64  
 4   Region_Code           float64
 5   Previously_Insured    int64  
 6   Vehicle_Age           object 
 7   Vehicle_Damage        object 
 8   Annual_Premium        float64
 9   Policy_Sales_Channel  float64
 10  Vintage               int64  
 11  Response              int64  
dtypes: float64(3), int64(6), object(3)
memory usage: 1.0+ GB


In [4]:
train.describe().T

,count,mean,std,min,25%,50%,75%,max
id,11504798.0,5.752398e+06,3.321149e+06,0.0,2876199.25,5752398.5,8628597.75,11504797.0
Age,11504798.0,3.838356e+01,1.499346e+01,20.0,24.00,36.0,49.00,85.0
Driving_License,11504798.0,9.980220e-01,4.443120e-02,0.0,1.00,1.0,1.00,1.0
Region_Code,11504798.0,2.641869e+01,1.299159e+01,0.0,15.00,28.0,35.00,52.0
Previously_Insured,11504798.0,4.629966e-01,4.986289e-01,0.0,0.00,0.0,1.00,1.0
Annual_Premium,11504798.0,3.046137e+04,1.645475e+04,2630.0,25277.00,31824.0,39451.00,540165.0
Policy_Sales_Channel,11504798.0,1.124254e+02,5.403571e+01,1.0,29.00,151.0,152.00,163.0
Vintage,11504798.0,1.638977e+02,7.997953e+01,10.0,99.00,166.0,232.00,299.0
Response,11504798.0,1.229973e-01,3.284341e-01,0.0,0.00,0.0,0.00,1.0


In [5]:
train.head()

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,0,Male,21,1,35.0,0,1-2 Year,Yes,65101.0,124.0,187,0
1,1,Male,43,1,28.0,0,> 2 Years,Yes,58911.0,26.0,288,1
2,2,Female,25,1,14.0,1,< 1 Year,No,38043.0,152.0,254,0
3,3,Female,35,1,1.0,0,1-2 Year,Yes,2630.0,156.0,76,0
4,4,Female,36,1,15.0,1,1-2 Year,No,31951.0,152.0,294,0


In [6]:
train = train.drop(columns = ['id'])
test = test.drop(columns = ['id'])

print(f'Shape of our data:{train.shape, test.shape}')

Shape of our data:((11504798, 11), (7669866, 10))


In [7]:
print(train['Response'].value_counts())

Response
0    10089739
1     1415059
Name: count, dtype: int64


**imbalance**

In [8]:
cols_ordinal = train.select_dtypes(include='object').columns
for column in cols_ordinal:
    print(column)
    print(train[column].value_counts(), '\n')

Gender
Gender
Male      6228134
Female    5276664
Name: count, dtype: int64 

Vehicle_Age
Vehicle_Age
1-2 Year     5982678
< 1 Year     5044145
> 2 Years     477975
Name: count, dtype: int64 

Vehicle_Damage
Vehicle_Damage
Yes    5783229
No     5721569
Name: count, dtype: int64 



**Vehicle_Age imbalance**

In [9]:
train_num_cols = train.select_dtypes(include=[np.number]).columns
train_num_cols[:7]

Index(['Age', 'Driving_License', 'Region_Code', 'Previously_Insured',
       'Annual_Premium', 'Policy_Sales_Channel', 'Vintage'],
      dtype='object')

# EDA

In [10]:
# profile = ProfileReport(train, title="Profiling Report")
# profile

# Preprocessing

In [11]:
X = train.drop("Response", axis=1)
y = train.Response

print(f'Shape X={X.shape}, y={y.shape}, test ={test.shape}')

Shape X=(11504798, 10), y=(11504798,), test =(7669866, 10)


In [12]:
def preprocess_df(X, Y, list_col, list_num):
    encoder = OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore')
    cat_col_names = list_col
    print(cat_col_names)
    encoder.fit(X[cat_col_names])
    X_train_ohe = encoder.fit_transform(X[cat_col_names])
    Y_test_ohe = encoder.transform(Y[cat_col_names])
    encoder_col_names = encoder.get_feature_names_out()
    X_train_ohe = pd.DataFrame(X_train_ohe, columns=encoder_col_names)
    Y_test_ohe = pd.DataFrame(Y_test_ohe, columns=encoder_col_names)
    
    num_col_names = list_num
    print(num_col_names)
    scaler = StandardScaler()
    encoder.fit(X[cat_col_names])
    X_train_scaled = scaler.fit_transform(X[num_col_names])
    Y_test_scaled = scaler.transform(Y[num_col_names])
    X_train_scaled = pd.DataFrame(X_train_scaled, columns=num_col_names)
    Y_test_scaled = pd.DataFrame(Y_test_scaled, columns=num_col_names)
    
    X = pd.concat([X_train_ohe, X_train_scaled], axis=1)
    Y = pd.concat([Y_test_ohe, Y_test_scaled], axis=1)
    return X, Y

In [13]:
for i in X.select_dtypes(include=[np.number]).columns:
    X[i+'+log'] = (X[i]+1).transform(np.log)
    test[i+'+log'] =(test[i]+1).transform(np.log)
    X[i+'+log1'] = (X[i]+1).transform(np.log1p)
    test[i+'+log1'] =(test[i]+1).transform(np.log1p)
    X[i+'+sqrt'] = (X[i]+1).transform(np.sqrt)
    test[i+'+sqrt'] =(test[i]+1).transform(np.sqrt)
    X[i+'+sqr'] = X[i]**2
    test[i+'+sqr'] = test[i]**2
    
print(f'Shape X={X.shape}, y={y.shape}, test ={test.shape}')

Shape X=(11504798, 38), y=(11504798,), test =(7669866, 38)


In [14]:
X, test = preprocess_df(X, 
                        test,
                        X.select_dtypes(include='object').columns,
                        X.select_dtypes(include=[np.number]).columns)

print(f'Shape X={X.shape}, y={y.shape}, test ={test.shape}')

Index(['Gender', 'Vehicle_Age', 'Vehicle_Damage'], dtype='object')
Index(['Age', 'Driving_License', 'Region_Code', 'Previously_Insured',
       'Annual_Premium', 'Policy_Sales_Channel', 'Vintage', 'Age+log',
       'Age+log1', 'Age+sqrt', 'Age+sqr', 'Driving_License+log',
       'Driving_License+log1', 'Driving_License+sqrt', 'Driving_License+sqr',
       'Region_Code+log', 'Region_Code+log1', 'Region_Code+sqrt',
       'Region_Code+sqr', 'Previously_Insured+log', 'Previously_Insured+log1',
       'Previously_Insured+sqrt', 'Previously_Insured+sqr',
       'Annual_Premium+log', 'Annual_Premium+log1', 'Annual_Premium+sqrt',
       'Annual_Premium+sqr', 'Policy_Sales_Channel+log',
       'Policy_Sales_Channel+log1', 'Policy_Sales_Channel+sqrt',
       'Policy_Sales_Channel+sqr', 'Vintage+log', 'Vintage+log1',
       'Vintage+sqrt', 'Vintage+sqr'],
      dtype='object')
Shape X=(11504798, 39), y=(11504798,), test =(7669866, 39)


In [15]:
invalid_chars = ['[', ']', '<']
for col in X.columns:
    if any(char in col for char in invalid_chars):
        print(f'Invalid character found in column name: {col}')     

Invalid character found in column name: Vehicle_Age_< 1 Year


In [16]:
X.columns = [col.replace('[', '').replace(']', '').replace('<', '') for col in X.columns]
test.columns = [col.replace('[', '').replace(']', '').replace('<', '') for col in test.columns]

# Threshold

In [17]:
def variance_threshold(df,th):
    var_thres=VarianceThreshold(threshold=th)
    var_thres.fit(df)
    new_cols = var_thres.get_support()
    return df.iloc[:,new_cols]

In [18]:
X_full = variance_threshold(X,0.04)
list_name = (X_full.columns)
test_full = test[list_name]

print(f'Shape X={X_full.shape}, y={y.shape}, test ={test_full.shape}')

Shape X=(11504798, 38), y=(11504798,), test =(7669866, 38)


# Oversempling, prepering ML

In [19]:
X_train, X_test, y_train, y_test = train_test_split(
    X_full, y, test_size=0.78, random_state=42, stratify = y)

In [20]:
sampler = RandomOverSampler(random_state=42)

X_train, y_train = sampler.fit_resample(X_train, y_train)

In [21]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_total = scaler.transform(test_full)

In [22]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver() # TPU detection
except ValueError:
    tpu = None
    gpus = tf.config.experimental.list_logical_devices("GPU")
    
if tpu:
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu,) 
    print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
elif len(gpus) > 1:
    strategy = tf.distribute.MirroredStrategy([gpu.name for gpu in gpus])
    print('Running on multiple GPUs ', [gpu.name for gpu in gpus])
elif len(gpus) == 1:
    strategy = tf.distribute.get_strategy() 
    print('Running on single GPU ', gpus[0].name)
else:
    strategy = tf.distribute.get_strategy() 
    print('Running on CPU')
print("Number of accelerators: ", strategy.num_replicas_in_sync)

Running on CPU
Number of accelerators:  1


# ML model

In [23]:
def create_model_2(input_shape):
    model = models.Sequential([
        layers.BatchNormalization(input_shape=input_shape),
        layers.Flatten(),
        layers.Dense(64, activation='relu', kernel_initializer=initializers.HeNormal(),
                     kernel_regularizer=regularizers.l2(0.4)), 
        layers.Dropout(0.3),
        layers.Dense(1, activation='sigmoid')
    ])
    return model

def create_xgboost_model():
    model = XGBClassifier(n_estimators=700, 
                          max_depth=6, 
                          learning_rate=0.1, 
                          objective='binary:logistic',
                          verbosity=0)
    return model

def create_catboost_model():
    model = CatBoostClassifier(iterations=700, 
                               depth=6, 
                               learning_rate=0.1, 
                               loss_function='Logloss', 
                               verbose=0)
    return model

def create_lightgbm_model():
    model = lgb.LGBMClassifier(n_estimators=700, 
                               max_depth=6, 
                               learning_rate=0.1, 
                               objective='binary',
                               verbose=-1)
    return model

folds = 5  
skf = StratifiedKFold(n_splits=folds, shuffle=True)

models_ensemble = []

for kfold, (train, val) in enumerate(skf.split(X_train, y_train)):
    tf.keras.backend.clear_session()

    model_xgboost = create_xgboost_model()
    model_dense = create_model_2(X_train.shape[1:])
    model_catboost = create_catboost_model()
    model_lightgbm = create_lightgbm_model()

    model_xgboost.fit(X_train[train], y_train[train], 
                      eval_set=[(X_train[val], y_train[val])],
                      early_stopping_rounds=10, verbose=False)
    models_ensemble.append(model_xgboost)
    val_pred_xgboost = model_xgboost.predict_proba(X_train[val])[:, 1]
    val_score_xgboost = roc_auc_score(y_train[val], val_pred_xgboost)
    print(f"Training on fold {kfold+1} out of {folds} val_score_xgboost", val_score_xgboost)
    
    model_dense.compile(loss='binary_crossentropy',
                    metrics=['binary_accuracy', 'Precision', 'Recall'],
                    optimizer=optimizers.Adam(learning_rate=1e-3))
    early_stopping = EarlyStopping(monitor='val_loss', patience=3)
    model_dense.fit(X_train[train], y_train[train], 
                epochs=10, verbose=10, 
                validation_data=(X_train[val], y_train[val]), 
                callbacks=[early_stopping])
    models_ensemble.append(model_dense)
    val_pred_2 = model_dense.predict(X_train[val]).flatten()
    val_score_2 = roc_auc_score(y_train[val], val_pred_2)
    print(f"Training on fold {kfold+1} out of {folds} val_score_model_dense", val_score_2)
    
    model_catboost.fit(X_train[train], y_train[train])
    models_ensemble.append(model_catboost)
    val_pred_3 = model_catboost.predict_proba(X_train[val])[:, 1]
    val_score_3 = roc_auc_score(y_train[val], val_pred_3)  
    print(f"Training on fold {kfold+1} out of {folds} val_score_model_catboost", val_score_3)
    
    model_lightgbm.fit(X_train[train], y_train[train], 
                       eval_set=[(X_train[val], y_train[val])],
                       eval_metric='logloss',
                       callbacks=[lgb.early_stopping(10)])
    models_ensemble.append(model_lightgbm)
    val_pred_lightgbm = model_lightgbm.predict_proba(X_train[val])[:, 1]
    val_score_lightgbm = roc_auc_score(y_train[val], val_pred_lightgbm)
    print(f"Training on fold {kfold+1} out of {folds} val_score_model_lightgbm", val_score_lightgbm)

def simple_ensemble_predictions(models, X_original):
    predictions = []
    for model in models:
        if isinstance(model, (CatBoostClassifier, XGBClassifier, lgb.LGBMClassifier)):
            pred = model.predict_proba(X_original)[:, 1]
        else:
            pred = model.predict(X_original)
        
        predictions.append(pred.flatten())
    
    predictions = np.array(predictions)
    averaged_predictions = np.mean(predictions, axis=0)
    return averaged_predictions

models_ensemble = [model_xgboost, model_dense, model_catboost, model_lightgbm]
ensemble_preds = simple_ensemble_predictions(models_ensemble, X_train[val])
ensemble_score = roc_auc_score(y_train[val], ensemble_preds)
print(f'Simple ensemble ROC AUC score: {ensemble_score}')

Training on fold 1 out of 5 val_score_xgboost 0.8820329614263278
Epoch 1/10
Epoch 2/10
Epoch 3/10
Epoch 4/10
27747/27747 ━━━━━━━━━━━━━━━━━━━━ 39s 1ms/step
Training on fold 1 out of 5 val_score_model_dense 0.842860504374239
Training on fold 1 out of 5 val_score_model_catboost 0.876608984743138
Training until validation scores don't improve for 10 rounds
Did not meet early stopping. Best iteration is:
[700]	valid_0's binary_logloss: 0.392056
Training on fold 1 out of 5 val_score_model_lightgbm 0.8809463643239364
Training on fold 2 out of 5 val_score_xgboost 0.8818645343173173
Epoch 1/10
Epoch 2/10
Epoch 3/10
Epoch 4/10
Epoch 5/10
Epoch 6/10
27747/27747 ━━━━━━━━━━━━━━━━━━━━ 34s 1ms/step
Training on fold 2 out of 5 val_score_model_dense 0.8415374358450955
Training on fold 2 out of 5 val_score_model_catboost 0.8763475972741901
Training until validation scores don't improve for 10 rounds
Did not meet early stopping. Best iteration is:
[700]	valid_0's binary_logloss: 0.392315
Training on fold

# Submit

In [24]:
# predict_tensor=model.predict(X_total)
ensemble_preds_new = simple_ensemble_predictions(models_ensemble,X_total)

239684/239684 ━━━━━━━━━━━━━━━━━━━━ 306s 1ms/step


In [25]:
sub = pd.read_csv('/kaggle/input/playground-series-s4e7/sample_submission.csv')
sub['Response'] = ensemble_preds_new #predict_tensor.flatten()
sub.to_csv('submission.csv', index=False)

In [26]:
sub.head()

,id,Response
0,11504798,0.052489
1,11504799,0.799513
2,11504800,0.716805
3,11504801,0.000577
4,11504802,0.330826
